# Pruning with our automatic strutured Pruning framework
Welcome to an end-to-end example for magnitude-based weight pruning

**Summary**

In this tutorial, you will:

* Train a tf.keras model for CIFAR10 from scratch.
* Fine tune the model by applying the pruning Framework and see the accuracy.





In [1]:
!git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
!echo $CWD
!cp -rf /content/Automatic-Structured-Pruning/* /content/

Cloning into 'Automatic-Structured-Pruning'...


$CWD


Der Befehl "cp" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [1]:
import pruning

## Train a model for CIFAR10 without pruning
Download and prepare the CIFAR10 dataset.
The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

Create the convolutional base
The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument input_shape to our first layer.

To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
#import matplotlib.pyplot as plt
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.
As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.


## Compile and train the model

In [3]:

comp = {
        "optimizer":'adam',
        "loss": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        "metrics": ['accuracy']}

model.compile(**comp)

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

Epoch 1/10
1563/1563 [==============================] - 11s 6ms/step - loss: 1.5520 - accuracy: 0.4282 - val_loss: 1.3089 - val_accuracy: 0.5187
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1599 - accuracy: 0.5853 - val_loss: 1.0514 - val_accuracy: 0.6306
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9992 - accuracy: 0.6476 - val_loss: 1.0259 - val_accuracy: 0.6350
Epoch 4/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8927 - accuracy: 0.6848 - val_loss: 0.9063 - val_accuracy: 0.6844
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8160 - accuracy: 0.7139 - val_loss: 0.9385 - val_accuracy: 0.6690
Epoch 6/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.7574 - accuracy: 0.7343 - val_loss: 0.8771 - val_accuracy: 0.6981
Epoch 7/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7018 - accuracy: 0.7553 - val_loss: 0.8364 - val_acc

In [4]:
model.save('my_model.h5') 

## Fine-tune pre-trained model with pruning
You will apply pruning to the whole model and see this in the model summary.

In this example, you prune the model with 10% Dense pruning and 10% Filter pruning.



In [7]:
dense_prune_rate=50
conv_prune_rate=50
prunemodel=pruning.prune_model('./my_model.h5',dense_prune_rate, conv_prune_rate,'L2', num_classes=10)

Finish with pruning
Conv2D
No Dense or Conv2D
Conv2D
No Dense or Conv2D
Conv2D
No Dense or Conv2D
Dense
Dense
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0      

d:\Users\Danie\Desktop\Hahn-Schickard\Automatic-Structured-Pruning\pruning.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return (np.array(layer_types), np.array(layer_params), layer_output_shape,


We see how we get less Parameter in the pruned Model

## Compile and re-train the model

In [9]:
prunemodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                1

In [8]:
compile = {
  "optimizer": 'adam',
  "loss": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  "metrics": 'accuracy'
}

prunemodel.compile(**compile,)

history = prunemodel.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

Epoch 1/10
1563/1563 [==============================] - 12s 7ms/step - loss: 1.2473 - accuracy: 0.5504 - val_loss: 1.0301 - val_accuracy: 0.6322
Epoch 2/10
1563/1563 [==============================] - 10s 7ms/step - loss: 0.9317 - accuracy: 0.6696 - val_loss: 0.9406 - val_accuracy: 0.6702
Epoch 3/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8491 - accuracy: 0.7009 - val_loss: 0.9222 - val_accuracy: 0.6775
Epoch 4/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7947 - accuracy: 0.7176 - val_loss: 0.9024 - val_accuracy: 0.6827
Epoch 5/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7585 - accuracy: 0.7299 - val_loss: 0.8750 - val_accuracy: 0.6953
Epoch 6/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7322 - accuracy: 0.7413 - val_loss: 0.9155 - val_accuracy: 0.6860
Epoch 7/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7002 - accuracy: 0.7531 - val_loss: 0.8843 - val_ac

Compare both models

In [10]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.8887 - accuracy: 0.7037
313/313 - 1s - loss: 0.8753 - accuracy: 0.7052


# Prune a model to a maximum accuracy loss

We define the compile arguments and the arguments fpr the Model.fit() function. 

In [14]:
# comp = {
#   "optimizer": 'adam',
#   "loss": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#   "metrics": 'accuracy'
# }



# automodel=pruning.pruning_for_acc('./my_model.h5', train_images, train_labels, comp, pruning_acc=None,
#                                   max_acc_loss=3, num_classes=10, label_one_hot=False, data_loader_path="./my_model.h5")